In [2]:
import cuspatial
import geopandas as gpd
import cudf
from numba import cuda
import numpy as np

In [101]:
#for cuXfilter, if remote
url = '***.***.***.***' # make this your notebook server's url
port = '****'          # set this to your notebook server's port

# Get your data

In [2]:
!curl https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip -o zones.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1001k  100 1001k    0     0   397k      0  0:00:02  0:00:02 --:--:--  397k


In [3]:
!curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2015-01.csv -o taxi2015.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1893M  100 1893M    0     0  4524k      0  0:07:08  0:07:08 --:--:-- 5142k   0     0  4302k      0  0:07:30  0:03:45  0:03:45 4392kM   58 1112M    0     0  4325k      0  0:07:28  0:04:23  0:03:05 4361k


In [4]:
!curl "https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=GeoJSON" > NYC_boroughs.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2881k    0 2881k    0     0   907k      0 --:--:--  0:00:03 --:--:--  907k


In [3]:
%%time
NYC_boroughs = gpd.read_file('NYC_boroughs.json')
NYC_boroughs.to_file('NYC_boroughs.shp')
NYC_gpu = cuspatial.read_polygon_shapefile('NYC_boroughs.shp')

CPU times: user 768 ms, sys: 265 ms, total: 1.03 s
Wall time: 1.04 s


# Let's check out the shape file

cuSpatial geometry uses a packed format. In this object, `NYC_gpu`, the first GPU-array in the tuple contains the feature positions `f_pos`. `Feature 0` is the first 24 rings, `Feature 1` is the set of rings from index 24-28, and so forth.

`r_pos` are the ring positions of each feature. `Ring 0` is the first 12 coordinate pairs, `Ring 1` is the coordinate pairs from index 12-511, and so forth.

Finally the `DataFrame` at position `2` in `NYC_gpu` are the 75519 coordinates used in all 5 features, and the 106 rings that compose them.


In [4]:
print(NYC_gpu)

(0     24
1     28
2     55
3     73
4    107
Name: f_pos, dtype: int32, 0         12
1        511
2        523
3        528
4        536
       ...  
102    70054
103    70300
104    75278
105    75291
106    75519
Name: r_pos, Length: 107, dtype: int32,                x          y
0     -73.896809  40.795808
1     -73.896785  40.796329
2     -73.897131  40.796798
3     -73.897883  40.797117
4     -73.898521  40.796936
...          ...        ...
75514 -73.907743  40.872846
75515 -73.907465  40.873547
75516 -73.907088  40.874326
75517 -73.906917  40.875056
75518 -73.906651  40.875753

[75519 rows x 2 columns])


In [5]:
t = NYC_gpu[2].iloc[511:522]
print(t)

             x          y
511 -73.898330  40.802413
512 -73.899387  40.801936
513 -73.899489  40.800901
514 -73.900037  40.800909
515 -73.899716  40.800799
516 -73.899787  40.799510
517 -73.900210  40.799264
518 -73.899025  40.799172
519 -73.898640  40.799101
520 -73.897985  40.799604
521 -73.896467  40.800790


You can see above the 11 vertices that make up `Feature 0`, `Ring 1`.

## Working with Polygons
Let's look at and dive into the elements of the read shapefile using cuSpatial

In [6]:
print("Polygon Bounds:" , NYC_gpu[0]) # uppper bound of the rings that make up the polygon (feature)
print("Last Vertex:" , NYC_gpu[1]) # this is the position of the last vertex in each ring

# You can get the lon/lat by 
print("Longitude: ", NYC_gpu[2]['x']) # prints lon
print("Latitude: " , NYC_gpu[2]['y']) # prints lat
      
NYC_gpu[2]['x'] = NYC_gpu[2]['x'].astype("float32")
NYC_gpu[2]['y'] = NYC_gpu[2]['y'].astype("float32")

Polygon Bounds: 0     24
1     28
2     55
3     73
4    107
Name: f_pos, dtype: int32
Last Vertex: 0         12
1        511
2        523
3        528
4        536
       ...  
102    70054
103    70300
104    75278
105    75291
106    75519
Name: r_pos, Length: 107, dtype: int32
Longitude:  0       -73.896809
1       -73.896785
2       -73.897131
3       -73.897883
4       -73.898521
           ...    
75514   -73.907743
75515   -73.907465
75516   -73.907088
75517   -73.906917
75518   -73.906651
Name: x, Length: 75519, dtype: float64
Latitude:  0        40.795808
1        40.796329
2        40.796798
3        40.797117
4        40.796936
           ...    
75514    40.872846
75515    40.873547
75516    40.874326
75517    40.875056
75518    40.875753
Name: y, Length: 75519, dtype: float64


In [7]:
NYC_gpu[1].head(25) #this set of rings are NYC_gpu[1][0] (the first polygon). This data is packed.  

0       12
1      511
2      523
3      528
4      536
5      561
6      581
7     1968
8     2065
9     2255
10    2260
11    2389
12    2496
13    2633
14    2638
15    2643
16    2649
17    2657
18    2664
19    2680
20    2685
21    2690
22    2714
23    8509
24    8556
Name: r_pos, dtype: int32

## Taxi Data
Let's import the taxi data.  Newer years have pickup/drop off location ids that you would have to cross correlate.  Older years, like 2015 give you a lon/lat values.  We'll be using those ideas

In [8]:
taxi2015 = cudf.read_csv("taxi2015.csv")
print(taxi2015.dtypes)

VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
pickup_longitude         float64
pickup_latitude          float64
RateCodeID                 int64
store_and_fwd_flag        object
dropoff_longitude        float64
dropoff_latitude         float64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtype: object


In [9]:
taxi2015.count()

VendorID                 12748986
tpep_pickup_datetime     12748986
tpep_dropoff_datetime    12748986
passenger_count          12748986
trip_distance            12748986
pickup_longitude         12748986
pickup_latitude          12748986
RateCodeID               12748986
store_and_fwd_flag       12748986
dropoff_longitude        12748986
dropoff_latitude         12748986
payment_type             12748986
fare_amount              12748986
extra                    12748986
mta_tax                  12748986
tip_amount               12748986
tolls_amount             12748986
improvement_surcharge    12748983
total_amount             12748986
dtype: int64

As you can see, the columns that we're interested in for our spatial analysis are `pickup_longitude`, `pickup_latitude`, `dropoff_longitude`, and `dropoff_latitude`.  However, they are `float64`. If we convert to `float32` we'll have GPU storage for twice as many features. Also, `float32` has plenty of precision for lon/lat computations.

In [10]:
taxi2015['pickup_longitude'] = taxi2015['pickup_longitude'].astype('float32')
taxi2015['pickup_latitude'] = taxi2015['pickup_latitude'].astype('float32')

taxi2015['dropoff_longitude'] = taxi2015['dropoff_longitude'].astype('float32')
taxi2015['dropoff_latitude'] = taxi2015['dropoff_latitude'].astype('float32')

## GPU Point-In-Polygon

- 12m pickup locations
- 12m dropoff locations
- 5 borough features
- 107 borough polygons
- 75518 borough coordinates

Now that we've collected the set of pickup locations and dropoff locations, we can use `cuSpatial.point_in_polygon_bitmap` to quickly determine which pickups and dropoffs occur in each borough. That is, 5 boroughs composed of a total of 107 polygons.

In [11]:
%%time
pickups = cuspatial.point_in_polygon_bitmap(taxi2015['pickup_longitude'] , taxi2015['pickup_latitude'], NYC_gpu[0], NYC_gpu[1], NYC_gpu[2]['x'], NYC_gpu[2]['y'])
dropoffs = cuspatial.point_in_polygon_bitmap(taxi2015['dropoff_longitude'] , taxi2015['dropoff_latitude'], NYC_gpu[0], NYC_gpu[1], NYC_gpu[2]['x'], NYC_gpu[2]['y'])

CPU times: user 27.4 s, sys: 48 ms, total: 27.5 s
Wall time: 27.5 s


In [12]:
pickups.head()

,in_polygon_24,in_polygon_28,in_polygon_55,in_polygon_73,in_polygon_107
0,False,False,False,False,True
1,False,False,False,False,True
2,False,False,False,False,True
3,False,False,False,False,True
4,False,False,False,False,True


In [13]:
dropoffs.head()

,in_polygon_24,in_polygon_28,in_polygon_55,in_polygon_73,in_polygon_107
0,False,False,False,False,True
1,False,False,False,False,True
2,False,False,False,False,True
3,False,False,False,False,True
4,False,False,False,False,True


We just computed point in polygon for 25 million points against 75.5k other points in `3.07s`. That's really fast. Simply computing the pairwise distance between all 25m points and the 75.5k polygon coordinates is `25000000*75500=2 trillion` comparisons. We're **NOT going to compare full CPU times**.  We will take a subset, to give you a taste though.

## CPU Benchmark

In the following section we'll pull a single 12 vertex polygon from the source data, create a `Shapely` polygon from it, and use Shapely to run point_in_polygon on a small subset of the 12m taxi pickup and dropoff locations.

In [17]:
t = NYC_gpu[2].iloc[511:522]
print(t)
NYC_shape = list(zip(t['x'].tolist(), t['y'].tolist()))
print(NYC_shape)

             x          y
511 -73.898331  40.802414
512 -73.899391  40.801937
513 -73.899490  40.800900
514 -73.900040  40.800907
515 -73.899719  40.800800
516 -73.899788  40.799511
517 -73.900208  40.799263
518 -73.899025  40.799171
519 -73.898643  40.799103
520 -73.897987  40.799603
521 -73.896469  40.800789
[(-73.89833068847656, 40.80241394042969), (-73.8993911743164, 40.801937103271484), (-73.89949035644531, 40.800899505615234), (-73.90003967285156, 40.800907135009766), (-73.89971923828125, 40.80080032348633), (-73.89978790283203, 40.79951095581055), (-73.90020751953125, 40.79926300048828), (-73.8990249633789, 40.799171447753906), (-73.89864349365234, 40.799102783203125), (-73.89798736572266, 40.79960250854492), (-73.89646911621094, 40.80078887939453)]


In [18]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

outcome = []
outcome_yes = 0
outcome_no = 0
random_1000 = taxi2015[['pickup_longitude', 'pickup_latitude']]#.iloc[
    #np.random.choice(np.arange(taxi2015.shape[0]), 1000000),:]
points = [Point(p) for p in list(zip(
    random_1000['pickup_latitude'].tolist(), random_1000['pickup_longitude'].tolist()))]
polygon = Polygon(NYC_shape)

In [19]:
%%time
for i in range(0, len(points)):
    point = points[i]
    if(polygon.contains(point)):
        outcome_yes += 1
    else:
        outcome_no += 1
    #outcome.append(point.contains(polygon)) #to keep track of the ride that would start in that polygon
print("yes: ", outcome_yes, "no: " , outcome_no)

yes:  0 no:  12748986
CPU times: user 35.8 s, sys: 428 ms, total: 36.2 s
Wall time: 36.2 s


Is it working?

In [20]:
center = polygon.centroid
print(polygon.contains(center))

True


Yup!


## Why? This needs an introduction.

Let's make all the booleans into integers (dirty way)

In [14]:
pickups = pickups*1
dropoffs = dropoffs*1

In [15]:
%%time
cols = ["in_polygon_24", "in_polygon_28", "in_polygon_55", "in_polygon_73", "in_polygon_107"] 
groups = pickups.groupby(cols, method='cudf')
# Define a function to apply to each group
def mult_add(in_polygon_24, in_polygon_28, in_polygon_55, in_polygon_73, in_polygon_107, polygon):
    for i in range(cuda.threadIdx.x, len(in_polygon_24), cuda.blockDim.x):
        if(in_polygon_24[i] == 1):
            polygon[i] = 24
        elif(in_polygon_28[i] == 1):
            polygon[i] = 28
        elif(in_polygon_55[i] == 1):
            polygon[i] = 55
        elif(in_polygon_73[i] == 1):
            polygon[i] = 73
        elif(in_polygon_107[i] == 1):
            polygon[i] = 107
        else:
            polygon[i] = 0

pickups = groups.apply_grouped(mult_add,
                              incols=["in_polygon_24", "in_polygon_28", "in_polygon_55", "in_polygon_73", "in_polygon_107"],
                              outcols={'polygon': np.int32},
                              # threads per block
                              tpb=8)
groups = dropoffs.groupby(cols, method='cudf')
dropoffs = groups.apply_grouped(mult_add,
                              incols=["in_polygon_24", "in_polygon_28", "in_polygon_55", "in_polygon_73", "in_polygon_107"],
                              outcols={'polygon': np.int32},
                              # threads per block
                              tpb=8)
print(pickups.head())
print(dropoffs.head())

     in_polygon_24  in_polygon_28  in_polygon_55  in_polygon_73  \
31               0              0              0              0   
61               0              0              0              0   
66               0              0              0              0   
157              0              0              0              0   
159              0              0              0              0   

     in_polygon_107  polygon  
31                0        0  
61                0        0  
66                0        0  
157               0        0  
159               0        0  
     in_polygon_24  in_polygon_28  in_polygon_55  in_polygon_73  \
31               0              0              0              0   
61               0              0              0              0   
66               0              0              0              0   
119              0              0              0              0   
157              0              0              0              0   

     in_

## Add the outputs to your taxi dataframe

In [16]:
taxi2015['pu_brgh'] = pickups['polygon']
taxi2015['do_brgh'] = dropoffs['polygon']

In [17]:
taxi2015.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pu_brgh,do_brgh
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,...,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05,107,107
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,...,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80,107,107
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,...,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80,107,107
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,...,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80,107,107
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,...,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30,107,107


In [18]:
taxi_cor = taxi2015.query('pu_brgh != 0 and do_brgh != 0')
taxi_cor = taxi_cor.query('do_brgh != 0')

In [86]:
taxi2015['all_within_boroughs'] = (taxi2015['pu_brgh'] !=0) & (taxi2015['do_brgh'] != 0)

In [84]:
taxi2015.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pu_brgh,do_brgh,in_boundary,in_boroughs
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,...,1.0,0.5,3.25,0.0,0.3,17.05,107,107,True,True
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,...,0.5,0.5,2.00,0.0,0.3,17.80,107,107,True,True
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,...,0.5,0.5,0.00,0.0,0.3,10.80,107,107,True,True
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,...,0.5,0.5,0.00,0.0,0.3,4.80,107,107,True,True
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,...,0.5,0.5,0.00,0.0,0.3,16.30,107,107,True,True


In [87]:
taxi2015['do_brgh'].value_counts()

107    11150551
55       642234
73       602716
0        282552
24        68489
28         2444
Name: do_brgh, dtype: int32

In [88]:
taxi2015['pu_brgh'].value_counts()

107    11612331
73       640695
0        256125
55       229807
24         9840
28          188
Name: pu_brgh, dtype: int32

In [89]:
taxi2015['all_within_boroughs'].value_counts()

True     12434189
False      314797
Name: all_within_boroughs, dtype: int32

In [ ]:
taxi2015['interborough']= (taxi2015[[]].assign(interborough=taxi_cor.pu_brgh != taxi_cor.do_brgh))

In [104]:
taxi2015['interborough'].value_counts()

False    11052069
True      1382120
Name: interborough, dtype: int32

The `True` and `False` values don't equal up to our full dataset numbers. The remainder, "None" type, will contain the extraborough transportaiton as well as the dirty data.  Good to look at with interest, but outside of our scope.  If you want to clean this data as well, you'll been boundary maps of th rest of the TriState area.  

# Visualization with Datashader
(You may remember me from other noteboks like...)  

We're going to do 8 visualizations
1) original data pickups
2) original data dropoffs
3) cuSpatial cleaned pickups
4) cuSpatial cleaned dropoffs
5) cleaned intraborough pickups
6) cleaned intraborough dropoffs
7) cleaned interborough pickups
8) cleaned interborough dropoffs

In [24]:
import cuxfilter
from bokeh import palettes
from cuxfilter.layouts import double_feature

In [94]:
from pyproj import Proj, Transformer
def makeXfilter(x, y):
    temp= cudf.DataFrame()
    transform_4326_to_3857 = Transformer.from_crs('epsg:4326', 'epsg:3857')
    temp['x'], temp['y'] = transform_4326_to_3857.transform(
                                                x.to_array(), y.to_array()
                                            )
    print(temp.head())
    return temp.x, temp.y

taxi2015['pu_x'], taxi2015['pu_y'] = makeXfilter(taxi2015['pickup_latitude'], taxi2015['pickup_longitude'])
taxi2015['do_x'], taxi2015['do_y'] = makeXfilter(taxi2015['dropoff_latitude'],taxi2015['dropoff_longitude'])
cux = cuxfilter.DataFrame.from_dataframe(taxi2015)

              x             y
0 -8.236963e+06  4.975553e+06
1 -8.237826e+06  4.971752e+06
2 -8.233561e+06  4.983296e+06
3 -8.238654e+06  4.970221e+06
4 -8.234434e+06  4.977363e+06
              x             y
0 -8.234835e+06  4.975627e+06
1 -8.237021e+06  4.976875e+06
2 -8.232279e+06  4.986477e+06
3 -8.238124e+06  4.971127e+06
4 -8.238108e+06  4.974457e+06


In [96]:
from bokeh.tile_providers import get_provider as gp
tile_provider = gp('CARTODBPOSITRON')

In [97]:
chart0 = cuxfilter.charts.scatter_geo(x='pu_x',
                                      y='pu_y',
                                      title='NYC Taxi Pickups',
                                      aggregate_fn='count',
                                      tile_provider=tile_provider, x_range=(-8267428.97,-8207328.23), y_range=(4935861.67,5000548.55))
chart1 = cuxfilter.charts.scatter_geo(x='do_x',
                                      y='do_y',
                                      title='NYC Taxi Dropoffs',
                                      aggregate_fn='count',
                                      tile_provider=tile_provider, x_range=(-8267428.97,-8207328.23), y_range=(4935861.67,5000548.55))
chart2 = cuxfilter.charts.multi_select('all_within_boroughs')
chart3 = cuxfilter.charts.multi_select('interborough')

In [98]:
d = cux.dashboard([chart0, chart1, chart2, chart3], layout=cuxfilter.layouts.feature_and_base, theme=cuxfilter.themes.dark, title= 'NYC TAXI DATASET')

In [99]:
d

#### cuxfilter DashBoard
    
    [title] Markdown(str)
    [chart1] Column(sizing_mode='scale_both', width=1600)
    [0] Bokeh(Figure)
    [chart2] Column(sizing_mode='scale_both', width=1600)
    [0] Bokeh(Figure)
    [widgets] Column
    [0] Column(css_classes=['non-handle-temp'], sizing_mode='scale_both')
        [0] FloatSlider(bar_color='#8735fb', end=12748986, name='Data Points selected', sizing_mode='scale_both', start=0, value=12748986, width=280)
    [1] Column(sizing_mode='scale_both', width=400)
        [0] MultiSelect(css_classes=['custom-dropdown'], height=200, name='all_within_boroughs', options=[False, True, ''], sizing_mode='scale_both', value=[''], width=280)
    [2] Column(sizing_mode='scale_both', width=400)
        [0] MultiSelect(css_classes=['custom-dropdown'], height=200, name='interborough', options=[None, False, True, ''], sizing_mode='scale_both', value=[''], width=280)

In [ ]:
d.show(url+':'+port) #if this fails, you didn't update your server url and port in cell 2!  Please go back and do it now, then rerun this cell

In [ ]:
d.stop()

# Looking for places of interest
So now that we have cleaned data of yellow cab trips, and verified it vusially using cuXFilter, we can use cuSpatial to see who is coming from where and why. One bit of curiousness is why, unlike the rest of the region, does Rockaway/Seaside have so many dropoffs, not almost no pickups.  This happens both with interborough and intraborough travel.  It's also Janurary, so nature tourism seems...unlikely.  Let's find out where these people are coming from.

To begin, we have to make a box with the latitudes and longitudes of our area of interest
40.633869, -73.748353
40.559571, -73.877628

In [106]:
pu_points_inside = cuspatial.window_points(
    40.559571, -73.877628,  40.633869, -73.748353, taxi2015['pickup_latitude'], taxi2015['pickup_longitude']
)
print(pu_points_inside.shape[0])

431


In [107]:
do_points_inside = cuspatial.window_points(
    40.559571, -73.877628,  40.633869, -73.748353, taxi2015['dropoff_latitude'], taxi2015['dropoff_longitude']
)
print(do_points_inside.shape[0])
print(do_points_inside)

1453
              x          y
0     40.625492 -73.780495
1     40.618229 -73.779617
2     40.596161 -73.766403
3     40.600056 -73.763321
4     40.597908 -73.776009
...         ...        ...
1448  40.593910 -73.755951
1449  40.596855 -73.772598
1450  40.599525 -73.782089
1451  40.574512 -73.858978
1452  40.590324 -73.797195

[1453 rows x 2 columns]


So, that's over 3x the pickup traffic.  Where is this influx coming from?  Let's merge back the other data on these points.

In [108]:
taxi2015['x'] = taxi2015['dropoff_latitude']
taxi2015['y'] = taxi2015['dropoff_longitude']
rwys = do_points_inside.merge(taxi2015)

Let's also do some time analysis and see the frequency of rides per day.  This requires us to modify the date a bit.

In [109]:
rwys['tpep_pickup_datetime']=rwys['tpep_pickup_datetime'].str.timestamp2int(format="%Y-%m-%d %H:%M:%S")

In [110]:
rwys['tpep_pickup_datetime']=rwys['tpep_pickup_datetime'].astype('datetime64[s]')
rwys['dom']=rwys['tpep_pickup_datetime'].dt.day
rwys['tod']= rwys['tpep_pickup_datetime'].dt.hour

In [111]:
day_freq = cudf.DataFrame()
day_freq = rwys['dom'].value_counts().reset_index()
print(day_freq)

    index  dom
0      20  135
1       8   97
2      12   97
3      13   94
4      14   84
5       9   78
6      10   63
7      11   58
8       7   51
9      22   48
10     21   45
11      4   44
12      6   44
13     17   44
14     30   43
15      1   41
16      5   39
17     15   39
18     19   38
19     29   38
20     18   37
21     28   37
22     16   35
23     24   34
24     31   32
25     25   31
26     23   29
27      2   28
28     26   27
29      3   26
30     27    9


In [ ]:
cux = cuxfilter.DataFrame.from_dataframe(day_freq)
chart1 = cuxfilter.charts.bar("index", "dom")
d = cux.dashboard([chart1], layout=cuxfilter.layouts.single_feature, theme=cuxfilter.themes.dark, title= 'NYC TAXI DATASET')
d.show(url+':'+port)

In [ ]:
d.stop()

In [75]:
print(rwys['pickup_latitude'], rwys['pickup_longitude'])

0       40.596291
1       40.598007
2       40.645374
3       40.743698
4       40.627956
          ...    
1284    40.644791
1285    40.597389
1286    40.593876
1287    40.644810
1288    40.744034
Name: pickup_latitude, Length: 1289, dtype: float32 0      -73.766205
1      -73.775963
2      -73.776764
3      -73.991951
4      -73.769829
          ...    
1284   -73.781883
1285   -73.772484
1286   -73.769615
1287   -73.777107
1288   -73.928352
Name: pickup_longitude, Length: 1289, dtype: float32


In [76]:
rwys['rwys_x'], rwys['rwys_y'] = makeXfilter(rwys['pickup_latitude'], rwys['pickup_longitude'])
rwys['rwys_x'].count()

              x             y
0 -8.211616e+06  4.952976e+06
1 -8.212703e+06  4.953227e+06
2 -8.212792e+06  4.960174e+06
3 -8.236746e+06  4.974610e+06
4 -8.212020e+06  4.957619e+06


1289

That's weird.  3/4 of the points dropped, even though they were corrected.  

In [118]:
rwys['rwys_x'], rwys['rwys_y'] = makeXfilter(rwys['pickup_latitude'], rwys['pickup_longitude'])
cux = cuxfilter.DataFrame.from_dataframe(rwys)
chart7 = cuxfilter.charts.multi_select('all_within_boroughs')
chart8 = cuxfilter.charts.multi_select('interborough')
chart10 = cuxfilter.charts.multi_select('dom')
chart11 = cuxfilter.charts.multi_select('tod')
chart9 = cuxfilter.charts.scatter_geo(x='rwys_x',
                                      y='rwys_y',
                                      title='Where are the drop offs coming from?',
                                         add_interaction=True , 
                                         color_palette=palettes.Purples9 , # use bokeh color palettes to get more pop out colors
                                         aggregate_col="tod" , aggregate_fn='max',
                                         tile_provider=tile_provider, x_range=(-8267428.97,-8207328.23), y_range=(4935861.67,5000548.55))

              x             y
0 -8.213270e+06  4.957294e+06
1 -8.236177e+06  4.974817e+06
2 -8.210027e+06  4.952227e+06
3 -8.212792e+06  4.960174e+06
4 -8.212785e+06  4.960163e+06


In [ ]:
d = cux.dashboard([chart7, chart8, chart9, chart10, chart11], layout=cuxfilter.layouts.single_feature, theme=cuxfilter.themes.dark, title= 'NYC TAXI DATASET')
d.show(url+':'+port)

In [43]:
d.stop()

And only because SOMEONE will ask "why didn't we show 'day of the month' and 'time of the day' on the large visualization?"...

In [120]:
taxi2015['tpep_pickup_datetime']=taxi2015['tpep_pickup_datetime'].str.timestamp2int(format="%Y-%m-%d %H:%M:%S")

In [121]:
taxi2015['tpep_pickup_datetime']=taxi2015['tpep_pickup_datetime'].astype('datetime64[s]')
taxi2015['dom']=taxi2015['tpep_pickup_datetime'].dt.day
taxi2015['tod']= taxi2015['tpep_pickup_datetime'].dt.hour

In [122]:
chart0 = cuxfilter.charts.scatter_geo(x='pu_x',
                                      y='pu_y',
                                      title='NYC Taxi Pickups',
                                      aggregate_fn='count',
                                      tile_provider=tile_provider, x_range=(-8267428.97,-8207328.23), y_range=(4935861.67,5000548.55))
chart1 = cuxfilter.charts.scatter_geo(x='do_x',
                                      y='do_y',
                                      title='NYC Taxi Dropoffs',
                                      aggregate_fn='count',
                                      tile_provider=tile_provider, x_range=(-8267428.97,-8207328.23), y_range=(4935861.67,5000548.55))
chart2 = cuxfilter.charts.multi_select('all_within_boroughs')
chart3 = cuxfilter.charts.multi_select('interborough')
chart4 = cuxfilter.charts.multi_select('dom')
chart5 = cuxfilter.charts.multi_select('tod')

In [ ]:
d = cux.dashboard([chart0, chart1, chart2, chart3], layout=cuxfilter.layouts.feature_and_base, theme=cuxfilter.themes.dark, title= 'NYC TAXI DATASET')
d.show(url+':'+port)

In [ ]:
d.stop()